# Analysis of Dog Ratings from users tweets
---


## Data Wrangling Report 
### By Samarth Agrawal


# Project Overview:

Goal of the project is to gather data from a variety of sources and in a variety of formats, assess its quality and tidiness, then clean it, analyze it and create visualizations using Python (and its libraries)

Our Dataset is the tweet archive of Twitter user @dog_rates, also known as `WeRateDogs`. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog

# Data Sources:

### Data is gathered from 3 different sources: 

1. CSV File : `twitter_archive.csv` <br>
    * This archive contains basic tweet data (tweet ID, timestamp, text, etc.) for all 5000+ of WeRateDogs as on August 1, 2017
    * I used pandas `read_csv()` to import csv data as pandas dataframe


2. Twitter API : `tweepy`
    * Basic data dump didn't contain two very important details about the tweet retweet count and favorite count. This additional data is gathered from Twitter's API
    * I have to first createa a developer account to get access to twitter api. learned how to access twitter api and stored all the data in a text file
    * Then loaded text file and learned how to parse json data to store in a pandas dataframe


3. File hosted on server : `image-predictions.tsv`
    * Every image in the WeRateDogs Twitter archive is scored through a Convolutional Neural Network that can classify breeds of dogs*. The results: a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction is provided in this file
    * I had to use `requests` library to access file kept on a server
    * I first downloaded the file on to my local system and then used pandas `read_csv()` method to load the data as pandas dataframe

# Data Assessing and Cleaning:

## Quality

### `twitter_archive` table

1. Some rows represent replies and not tweets: 78 records
    * If the variable `in_reply_to_status_id` is populated it means that row is a reply. I deleted those rows
    
    
2. Some rows represent retweets and not tweets: 181 records. 
    * If the variable `retweeted_status_id` is populated it means that row is a reply. I deleted those rows


3. Tweet id is loaded as `int` type. Since this is an id variable we can make it `str` type
    * I used `astype(str)` method to convert into str type


4. Missing values in Dog Stages - Doggo, pupper, etc is recorded as None string instead of pandas standard NaN. 
    * This issue was taken care when I solved for the structural problem. Instead of having these columns separately I created a new column called `dog_stages` and removed these columns


5. Wrong datatype : variable timestamp got stored as object instead of date. 
    * Used pandas `to_datetime()` method to convert this into datetime variable


6. If rating contained decimals it's not parsed correctly Ex, 9.75/10, 11.26/10
    * Wrote a new regex code that will take care of decimal inputs. `str.extract('(\d+(\.\d+)?/\d+)')`
    * Kept the original rating_numerator and rating_denominator for comparison and then deleted the old ones and kept only the new columns


7. It was mentioned that Rating Denominator has to be 10. But it was found in some records that the value of rating_denominator was not 10
    * Users were asked to rate on the scale of 10 so if denominator is not 10 they are distorting the basis of comparison
    * Used value_counts() to check how many such records were present
    * First created a list of index values for all such records were denominator!=10 and then used pandas `drop()` methond drop(index=drop_idx) method to delete those rows


8. In one of the tweet a reference to time 24/7 was made but it was parsed as rating in the dataset
    * This issue was corrected along with the previous one because denominator of this is not equal to 10


9. There are several columns that are not needed for analysis: 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'
    * First created a list of all the columns to be deleted. let's call it drop_cols and then used pandas `drop()` method drop(columns = drop_cols)


10. Name is not parsed correct. Name of `a`, `an`, `my` etc because the way some people tweet is `This is a`
    * I'm not correcting for this issue yet because I'm not doing any analysis that's based on the name of the dog



### `image_pred` table
11. Tweet id is loaded as int type. Since this is an id variable we can make it str type
    * I used `astype(str)` method to convert into str type


12. There are several columns that are not needed for analysis : 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'
    * First created a list of all the columns to be deleted. let's call it drop_cols and then used pandas `drop()` method drop(columns = drop_cols)
    
    

### `tweet_df` table
13. Tweet id is loaded as int type. Since this is an id variable we can make it str type
    * I used `astype(str)` method to convert into str type
    
    

## Tidiness (Structural) Issues

1. Dog Stages should not be in separate columns
    * This was the first issue i corrected
    * First I created a function `dog_stages(row)` that took row as input and checked if column doggo was populated then it will return doggo as value to the new column `dog_stages` and so on
    * Then used pandas `apply()` function to create a new column `dog_stages'


2. Retweet count and favorite count should be in same dataframe as twitter_archive
    * I used pandas `merge()` method on twitter_archive and did `left` join on tweet_df to create a new dataframe `twitter_archive_master` 
    
    
3. Neural Network output values should in same dataframe as twitter_archive
    * I used pandas `merge()` method on twitter_archive_master and did `left` join on image_pred to create a new dataframe `twitter_archive_master` 
    
    

## Missing Data

1. `image_pred` table : Missing 281 records (2075 instead of 2356)
1. `tweet_df` table : Missing 23 records (2333 instead of 2356)

<b>Since these are user provided data, we can't use any imputation technique to impute the values missing in these records

# Storing the Clean Data:

* After all the cleaning issues, final dataset was stored in a csv file `twitter_archive_master.csv` for future use using the pandas `to_csv()` method